In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e8/sample_submission.csv
/kaggle/input/playground-series-s5e8/train.csv
/kaggle/input/playground-series-s5e8/test.csv


In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss

from xgboost import XGBClassifier

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

train_df = pd.read_csv('/kaggle/input/playground-series-s5e8/train.csv')
test_df = pd.read_csv('/kaggle/input/playground-series-s5e8/test.csv')

In [5]:
target = 'y'
feature_cols = [col for col in train_df.columns if target not in col]
categorical_columns = ['job','marital','education','default','housing','loan','contact','month','poutcome']
print(feature_cols)

['id', 'age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'month', 'duration', 'campaign', 'previous', 'poutcome']


In [6]:
# Create a copy of the dataframes to avoid modifying originals
train_scaled = train_df.copy()
test_scaled = test_df.copy()

# Select columns to scale (exclude target column)
# - Only consider numeric columns
numeric_cols = train_scaled.select_dtypes(include=["number"]).columns
numeric_cols = [col for col in numeric_cols if col != target]
scaler = StandardScaler()

# Fit the scaler on the training data (numeric columns)
scaler.fit(train_scaled[numeric_cols])

# Transform both train and test data
train_scaled[numeric_cols] = scaler.transform(train_scaled[numeric_cols])
test_scaled[numeric_cols] = scaler.transform(test_scaled[numeric_cols])

In [ ]:
# Create a copy of train and test dataframes to avoid modifying original dataframes
train_encoded = train_scaled.copy()
test_encoded = test_scaled.copy()

# Initialize label encoder
le = LabelEncoder()

# Apply label encoding to each categorical column
for column in categorical_columns:
    print(f'Encoding: {column} ...')
    # Fit the label encoder on the train data
    le.fit(train_encoded[column])
    
    # Transform both train and test data using the same encoder
    train_encoded[column] = le.transform(train_encoded[column])
    if column in test_encoded.columns:
        # Handle cases where test set may have unseen labels by using fillna
        test_encoded[column] = test_encoded[column].map(lambda s: le.transform([s])[0] if s in le.classes_ else None)
        test_encoded[column].fillna(-1, inplace=True)
        test_encoded[column] = test_encoded[column].astype(int)

Encoding: job ...
Encoding: marital ...
Encoding: education ...
Encoding: default ...


In [ ]:
# Determine feature set
X = train_encoded[feature_cols]
X_test = test_encoded[feature_cols]

# Extract target values
y = train_encoded[target]

# Initialize the model with the specified parameters
model = XGBClassifier()

In [ ]:
# run a grid search
param_grid = {
    "n_estimators": [100, 200],
    "max_depth": [3, 6, 9],
    "learning_rate": [0.01, 0.1, 0.2]
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring="roc_auc", cv=5, verbose=1)

grid_search.fit(X, y)